# Data cleaning and merging:

In [1]:
# Necessary imports:
import pandas as pd
import re

### Loading the two CSV files as dataframes:

In [2]:
df_results = pd.read_csv("../data/cleaned/cleaned_result.csv")
df_tables = pd.read_csv("../data/raw/raw_tables.csv")

In [3]:
df_results

,Detailed Result,General Result,Result as Number
0,2nd Try Success,Success,2
1,3rd Try Success,Success,3
2,3rd Try Success,Success,3
3,3rd Try Success,Success,3
4,2nd Try Success,Success,2
...,...,...,...
593,2nd Try Success,Success,2
594,1st Try Success,Success,1
595,2nd Try Success,Success,2
596,Failed,Failed,0


In [4]:
df_tables

,Episode #,Song Questions + Snack Time Game,Air Date,Month,Year
0,1,BTS - DNA,"April 7, 2018",April,2018
1,1,Bolbbalgan4 - Fight Day,"April 7, 2018",April,2018
2,1,Seventeen - Very Nice,"April 7, 2018",April,2018
3,2,Chakra - Hey U,"April 14, 2018",April,2018
4,2,Girls' Generation-TTS - Adrenaline,"April 14, 2018",April,2018
...,...,...,...,...,...
914,308,JuJu Club - Essay Love,"March 30, 2024",March,2024
915,308,If It's the End Quiz,"March 30, 2024",March,2024
916,308,Beenzino (feat. Cautious Clay) - Travel Again,"March 30, 2024",March,2024
917,309,TBA - TBA,"April 6, 2024",April,2024


### Declaring two functions that will determine if the current row in the "Song Question + Snack Time Game" in the For Loop is a Song or not:

In [5]:
def song_catcher(text: str) -> bool:
    """Accepts a string and determines if its a song or not based on the format "String - String".

    Args:
        text (str): A string to be determine if its a song or not.

    Returns:
        bool: True -> It is a song.\n\t\tFalse -> Not a song.
    """
    pattern = r"[a-zA-Z0-9`~!@#$%^&*)(=+_\}{';:.>,<?/-Â] - [a-zA-Z0-9`~!@#$%^&*)(=+_\}{';:.>,<?/-Â]|[a-zA-Z0-9`~!@#$%^&*)(=+_\}{';:.>,<?/-Â] -[a-zA-Z0-9`~!@#$%^&*)(=+_\}{';:.>,<?/-Â]"
    is_it_a_song = False
    
    matched = re.findall(pattern, text)
    if matched:
        is_it_a_song = True
        return is_it_a_song
    else:
        return is_it_a_song

def song_remover(text: str) -> bool:
    """Accepts a string and determines if its a song to be 
    remove or not since not all song has a result in it.

    Args:
        text (str): A string to be determine if its to be remove or not.

    Returns:
        bool: True -> Remove the song.\n\t\tFalse -> Don't remove the song.
    """
    # This list contains the songs with no Result in it.
    song_list = [
        "Jin (BTS) - Super Tuna",
        "Badkiz - Ear Attack",
        "DJ DOC - Let's Go to the Beach",
        "BTS - Airplane pt.2",
        "Deux - We Are",
        "TXT - New Rules",
        "Turbo - Only Seventeen",
        "Lee Seung-yoon - Some Some Some",
        "TBA - TBA"
    ]
    
    remove_song = False
    
    if text in song_list:
        remove_song = True
        return remove_song
    
    return remove_song

### Now, we iterate the "df_tables" to get the data on each row. Then, save the current data of the column "Song Question + Snack Time Game" in "current_song" to be tested by the two functions that was declared above this. If it is a song with Result, the whole row will be appended to the initially empty list "songs_handler" as a dictionary. After the For Loop has finished, it will then saved as a dataframe "df_table_songs".

In [6]:
songs_handler = []

for _, row in df_tables.iterrows():
    current_row = row.copy()
    current_song = str(current_row["Song Questions + Snack Time Game"])
    
    remove_song = song_remover(current_song)
    if remove_song is True:
        continue
    
    is_it_a_song = song_catcher(current_song)
    if is_it_a_song is True:
        songs_handler.append(current_row.to_dict())

df_table_songs = pd.DataFrame(songs_handler)

In [7]:
df_table_songs

,Episode #,Song Questions + Snack Time Game,Air Date,Month,Year
0,1,BTS - DNA,"April 7, 2018",April,2018
1,1,Bolbbalgan4 - Fight Day,"April 7, 2018",April,2018
2,1,Seventeen - Very Nice,"April 7, 2018",April,2018
3,2,Chakra - Hey U,"April 14, 2018",April,2018
4,2,Girls' Generation-TTS - Adrenaline,"April 14, 2018",April,2018
...,...,...,...,...,...
593,306,Lee Hyo-ri - Anyclub,"March 16, 2024",March,2024
594,307,ZE:A - Variety of Ways,"March 23, 2024",March,2024
595,307,Cherry Filter - Supermarket,"March 23, 2024",March,2024
596,308,JuJu Club - Essay Love,"March 30, 2024",March,2024


### Now, we can see that this cleaned table that the pandas has scrape has now the same number of rows as the "df_results" from Selenium. The data cleaning is a success.
### Its now time to merge the two dataframes and do some column ordering / formatting:

In [8]:
df_merged = pd.concat([df_table_songs, df_results], axis=1)
columns_inorder = ["Episode #", "Song Questions + Snack Time Game", "Air Date", "Month", "Year", "Detailed Result", "General Result", "Result as Number"]
df_merged[columns_inorder] = df_merged[columns_inorder].astype("object")

In [9]:
df_merged

,Episode #,Song Questions + Snack Time Game,Air Date,Month,Year,Detailed Result,General Result,Result as Number
0,1,BTS - DNA,"April 7, 2018",April,2018,2nd Try Success,Success,2
1,1,Bolbbalgan4 - Fight Day,"April 7, 2018",April,2018,3rd Try Success,Success,3
2,1,Seventeen - Very Nice,"April 7, 2018",April,2018,3rd Try Success,Success,3
3,2,Chakra - Hey U,"April 14, 2018",April,2018,3rd Try Success,Success,3
4,2,Girls' Generation-TTS - Adrenaline,"April 14, 2018",April,2018,2nd Try Success,Success,2
...,...,...,...,...,...,...,...,...
593,306,Lee Hyo-ri - Anyclub,"March 16, 2024",March,2024,2nd Try Success,Success,2
594,307,ZE:A - Variety of Ways,"March 23, 2024",March,2024,1st Try Success,Success,1
595,307,Cherry Filter - Supermarket,"March 23, 2024",March,2024,2nd Try Success,Success,2
596,308,JuJu Club - Essay Love,"March 30, 2024",March,2024,Failed,Failed,0


### We can see that it has been merged successfully. Now saving the dataframes "df_table_songs" and "df_merged" as CSV Files to their respective folders:

In [10]:
df_table_songs.to_csv("../data/cleaned/cleaned_tables.csv", index=False)
df_merged.to_csv("../data/merged/data_merged.csv", index=False)

## To the final step -> "data_visualization.ipynb"